Define PCA transform to keep PC's accounting for 95% of data variance

In [1]:
def PCATransform(data):
    pca = PCA()
    pca.fit(data)

    toKeep = 0
    totalVar = 0
    for var in pca.explained_variance_ratio_:
        if totalVar < .95:
            totalVar += var
            toKeep += 1
        else:
            break

    print("Top {} components capture {:.2f}% of the data".format(toKeep,totalVar*100))

    outData = PCA(n_components = toKeep).fit_transform(data)
    outData = pd.DataFrame(data=outData)
    print("{} -> {}".format(data.shape,outData.shape))
    
    return outData

Popular data will be seperated from raw data. As regression will be used to predict popular chart rankings, all models will be run on popular data only. Datasets used will be all popular data, and the 5 genre breakdowns.

The following data variants will be created per dataset:
1. NOCOR
   - Dropping highly correlated variables
2. NODUR
   - Dropping duration
     - *Duration was found to be overly valued during classification, but intution says it should not be particularly relevent to a songs popularity. Dropping to observe differences*
3. RECENT
  - Subset of only most recent 3 years of data
    - *Used to see if stronger trends emerge when looking at only recent data. Assumption here is that song trends change somewhat frequently and past year data may throw off results when grouped together*
4. PCA
  - PCA performed on continuous variables, along with dummy variables unchanged

In [4]:
import pandas as pd
# Initial data import
rawData = pd.read_csv("./data/rawData_final.csv")

popData = {}

# Split popular data from raw and seperate chartrank (target)
popData['raw'] = rawData[rawData.popular == 1].drop('popular', axis=1)
popData['all_target'] = popData['raw'].chartrank
popData['all'] = popData_raw.drop('chartrank',axis=1)
popData['all'] = pd.get_dummies(popData['all'], columns=['genre'])

NameError: name 'popData_raw' is not defined

Pull 3 most recent years: 2018, 2019, 2020

In [ ]:
recentYear = 2018
popData['recent'] = popData['all'][popData['all'].year >= recentYear]
popData['recent'] = pd.get_dummies(popData['recent'], columns=['year'])
popData['recent_target'] = popData['all_target'][popData['recent'].index]

Get dummy vars for years

In [ ]:
popData['all'] = pd.get_dummies(popData['all'], columns=['year'])

Drop correlated vars
### <font color='red'>Going to be calculated using grid search after model grid search done on corresponding 'all' data - using selectKBest from sklearn - different score function depending on classificition vs regression vs clustering</font>

In [ ]:
selectBestK = FS.SelectKBest(score_func=f_regression, k=k) # k is gridsearch func, f_regression for regression scoring
selectBestK.fit(popData['all'],popData['all_target'])

Drop duration

In [ ]:
popData['nodur'] = popData['all'].drop('duration_ms',axis=1)

PCA on continuous, with original dummies

In [ ]:
dummies = ['time_signature','explicit','key','mode','year','genre']
continuousVars = [x for x in popData['all'].columns]

for dummy in dummies:
    continuousVars = [x for x in continuousVars if dummy not in x]
    
print("Continuous variables:")
for var in continuousVars:
    print("  ",var)
print()
    
dummyVars = [x for x in popData['all'].columns if x not in continuousVars]

print('-------Popular PCA-------')
popData['pca'] = PCATransform(popData['all'][continuousVars])
popData['pca'] = pd.concat([popData['pca'], popData['all'][dummyVars].reset_index(drop=True)], axis=1)
print()
popData['pca'].info()

Split by genre, create variants per

In [ ]:
# Genre split
genres = np.unique(popData_raw.genre.to_numpy())

popGenre = {}
for genre in genres:
    raw = popData['raw'][popData['raw'].genre == genre]
    target = raw.chartrank
    gAll = raw.drop(['genre','chartrank'],axis=1)
    
    # Recent 3 years
    recent = gAll[gAll.year >= recentYear]
    recent = pd.get_dummies(recent, columns=['year'])
    recent_target = target[recent.index]
    
    # Dummy year
    gAll = pd.get_dummies(gAll, columns=['year'])
    
    # Drop correlated
    nocor = gAll.drop(correlatedVars,axis=1)
    
    # Drop duration
    nodur = gAll.drop('duration_ms',axis=1)
    
    dummyVars = [x for x in dummyVars if x in gAll.columns]
    
    # PCA
    print('-------{} PCA-------'.format(genre))
    pca = PCATransform(gAll[continuousVars])
    pca = pd.concat([pca, gAll[dummyVars].reset_index(drop=True)], axis=1)
    print()
    
    popGenre[genre] = {"all":gAll,"all_target":target,"recent":recent,"recent_target":recent_target,"nocor":nocor,"nodur":nodur,"pca":pca}